# Custom loss trial for FlowNet

In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt

In [11]:
class CustomModel(tf.keras.Model):

    def __init__(self):
        super(CustomModel, self).__init__()
        self.input_layer = keras.layers.Lambda(lambda x: K.log( x+2  ) )

    def findGrad(self,func,argm):
        return keras.layers.Lambda(lambda x: K.gradients(x[0],x[1])) ([func,argm])
    
    def call(self, inputs):
        Z = self.input_layer(inputs)
        Z_1 = self.findGrad(Z,inputs)
        #     return self.dense2(x)
        #     print("\n\n\n this is the answer:",self.square_layer(inputs))
        #     print("hre is a break")
        return Z,Z_1


custom_model = CustomModel()

In [12]:
x = np.array([[1.],
            [2.],
            [3.]],dtype=np.float32)
X = x 

Y_1 = np.array([[0.],
            [0.],
            [0.]])
Y_2 = np.array([[1.],
            [1.],
            [1.]])
Y = np.concatenate((Y_1,Y_2),axis=1)
Y = np.array([[1., 1.],
       [0., 0],
       [1., 1.]],dtype=np.float32)

In [13]:
y_pred_test = custom_model.predict(x)
custom_model.predict(x)

(array([[1.0986123],
        [1.3862944],
        [1.609438 ]], dtype=float32),
 [array([[0.33333334],
         [0.25      ],
         [0.2       ]], dtype=float32)])

In [14]:
def custom_loss(y_true, y_pred):
    y_true_act = y_true[:,:-1]
    y_pred_u = y_pred
#     print(y_true_act)
    at_boundary = tf.cast(y_true[:,-1:],bool)
    print(at_boundary)
#     pred_sq_error = tf.square(y_true_act-y_pred_u)
    pred_sq_error = tf.abs(y_true_act-y_pred_u)
#     boundary_error = tf.zeros(shape=pred_sq_error.shape)
    return tf.where(at_boundary,0.,pred_sq_error)

In [15]:
custom_model.compile(loss=custom_loss)

In [16]:
custom_model.fit(x=X,y=Y)

Tensor("custom_loss/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss_1/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss_1/Cast:0", shape=(None, 1), dtype=bool)
1/1 [==============================] - 0s 1ms/step - loss: 0.5454 - output_1_loss: 0.4621 - output_2_1_loss: 0.0833


In [6]:
custom_loss(Y,y_pred_test)

tf.Tensor(
[[ True]
 [False]
 [ True]], shape=(3, 1), dtype=bool)


<tf.Tensor: shape=(3, 1), dtype=float32, numpy=
array([[0.       ],
       [1.3862944],
       [0.       ]], dtype=float32)>

In [7]:
custom_model.compile(loss=custom_loss)

In [8]:
custom_model.fit(x=X,y=Y)

Tensor("custom_loss/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss_1/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss/Cast:0", shape=(None, 1), dtype=bool)
Tensor("custom_loss_1/Cast:0", shape=(None, 1), dtype=bool)
1/1 [==============================] - 0s 1ms/step - loss: 0.6031 - output_1_loss: 0.5365 - output_2_1_loss: 0.0667
